잠재요인기반 추천시스템 ALS 모델 만들기

In [4]:
from pyspark.sql import SparkSession

MAX_MEMORY = '5g'
spark = SparkSession.builder.appName("241212_01_MLlib_als")\
            .config("spark.executor.memory", MAX_MEMORY) \
            .config("spark.driver.memory", MAX_MEMORY) \
            .getOrCreate()

In [3]:
spark.stop()

# data 적재

In [5]:
rating_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/ratings.csv')

In [6]:
rating_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
rating_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



# 전처리 

- timestamp 제거

In [8]:
rating_df = rating_df.select(['userId', 'movieId', 'rating'])

In [13]:
rating_df.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
+------+-------+------+
only showing top 5 rows



# 데이터 세트 분할

In [9]:
train_ratio = 0.8
test_ratio = 0.2

train_df, test_df = rating_df.randomSplit([train_ratio, test_ratio], seed=42)

# ALS 모델 객체 생성

In [22]:
from pyspark.ml.recommendation import ALS

als = ALS(
    maxIter=5,
    regParam=0.1,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)

# 모델의 학습

In [23]:
als_model = als.fit(train_df)

# 예측 확인

In [25]:
#userid, movieid > rating 
predictions = als_model.transform(test_df)
predictions.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
| 26480|    148|   2.0| 1.9371556|
|151614|    148|   1.0| 2.7611623|
| 28229|    148|   1.0| 2.4541242|
|  6491|    148|   4.0| 2.4168284|
| 14831|    148|   3.0| 2.7606502|
+------+-------+------+----------+
only showing top 5 rows



In [27]:
predictions.select("rating", "prediction").describe().show()

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           4998109|           4998109|
|   mean|3.5341648211353536|3.3860397680020453|
| stddev|1.0609230261741123|0.6349173760838954|
|    min|               0.5|         -3.052301|
|    max|               5.0|          6.618273|
+-------+------------------+------------------+



# ALS 모델의 하이퍼파라미터 조정

coldStartStrategy = 'drop' > 예측에 충분한지 않은 컬럼의 데이터의 경우 'drop'

학습한 모델의 속성을 변경 > 다시 예측을 한 경우에도 조정이 안됨 > 학습부터 다시 실행

In [19]:
als.setColdStartStrategy('drop')

ALS_6c14407100dc

In [20]:
#userid, movieid > rating 
predictions = als_model.transform(test_df)
predictions.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
| 26480|    148|   2.0| 1.9371556|
|151614|    148|   1.0| 2.7611623|
| 28229|    148|   1.0| 2.4541242|
|  6491|    148|   4.0| 2.4168284|
| 14831|    148|   3.0| 2.7606502|
+------+-------+------+----------+
only showing top 5 rows



In [21]:
predictions.select("rating", "prediction").describe().show()

+-------+------------------+----------+
|summary|            rating|prediction|
+-------+------------------+----------+
|  count|           5000974|   5000974|
|   mean|3.5338406078495908|       NaN|
| stddev|1.0610380809122413|       NaN|
|    min|               0.5| -3.052301|
|    max|               5.0|       NaN|
+-------+------------------+----------+



# 평가

RMSE 측정

In [26]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [28]:
rmse = evaluator.evaluate(predictions)
rmse

0.8171137812944954

In [ ]:
# 활용

1. 사용자id > 추천목록
2. 영화id > 사용자목록

In [ ]:
als_model.getRecommentForAllUsers(3)  #user별로 추천할 영화 3개를 골라준다.

In [ ]:
als_model.recommandForAllItems(3) #item별로 추천할 유저 3명을 골라준다

In [31]:
from pyspark.sql.types import IntegerType

In [34]:
user_list = [65, 78, 81]
user_df = spark.createDataFrame(user_list, IntegerType()).toDF("userId")
user_df.show()

+------+
|userId|
+------+
|    65|
|    78|
|    81|
+------+



In [36]:
user_recommend_movies = als_model.recommendForUserSubset(user_df, 3)
user_recommend_movies.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    65|[{194434, 7.40906...|
|    78|[{177209, 7.09564...|
|    81|[{134673, 4.64257...|
+------+--------------------+



In [ ]:
# movie-list load

In [37]:
movie_file = 'data/movies.csv'
movies_df = spark.read.csv(movie_file, inferSchema=True, header=True)

In [38]:
movies_df.show(3)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows



In [39]:
#65번 사용자의 추천영화목록
movies_list = user_recommend_movies.collect()[0].recommendations
movies_list

[Row(movieId=194434, rating=7.4090657234191895),
 Row(movieId=205277, rating=7.142080307006836),
 Row(movieId=120272, rating=6.9019389152526855)]

In [40]:
rec_df = spark.createDataFrame(movies_list)
rec_df.show()

+-------+------------------+
|movieId|            rating|
+-------+------------------+
| 194434|7.4090657234191895|
| 205277| 7.142080307006836|
| 120272|6.9019389152526855|
+-------+------------------+



In [ ]:
# 영화정보와 추천영화 목록을 조인

In [41]:
rec_df.createOrReplaceTempView('recommend')
movies_df.createOrReplaceTempView('movies')

In [42]:
query = '''
select * 
from movies join recommend on movies.movieId = recommend.movieId
order by rating desc
'''

In [43]:
recommend_movies = spark.sql(query)

In [44]:
recommend_movies.show()

+-------+-----------------+--------------------+-------+------------------+
|movieId|            title|              genres|movieId|            rating|
+-------+-----------------+--------------------+-------+------------------+
| 194434|Adrenaline (1990)|  (no genres listed)| 194434|7.4090657234191895|
| 205277|Inside Out (1991)|Comedy|Drama|Romance| 205277| 7.142080307006836|
| 120272| Get Crazy (1983)|              Comedy| 120272|6.9019389152526855|
+-------+-----------------+--------------------+-------+------------------+



In [ ]:
spark.stop()